### Load libraries 

In [ ]:
import neurokit2 as nk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Load data 
note: this is still the old code and not updated with Heejung's code yet

In [ ]:
run_num = 2

spacetop_data, spacetop_samplingrate = nk.read_acqknowledge('/Users/isabelneumann/Desktop/CANlab/Heejung/Neurokit/data/biopac_sub-0051-0053/sub-0051/ses-03/SOCIAL_spacetop_sub-0051_ses-03_task-social_ANISO.acq')
print(spacetop_data.columns)
bdf = pd.read_csv('/Users/isabelneumann/Desktop/CANlab/Heejung/Neurokit/data/behavioral_sub-0051-0053/sub-0051/ses-03/sub-0051_ses-03_task-social_run-02-pain_beh.csv')

### Organize data 
note: this is still the old code and not updated with Heejung's code yet, therefore there are not too many comments or markdowns 

In [ ]:
mid_val = (np.max(spacetop_data['trigger']) - np.min(spacetop_data['trigger']))/2
spacetop_data.loc[spacetop_data['trigger'] > mid_val, 'fmri_trigger'] = 5
spacetop_data.loc[spacetop_data['trigger'] <= mid_val, 'fmri_trigger'] = 0

start_df = spacetop_data[spacetop_data['fmri_trigger'] > spacetop_data['fmri_trigger'].shift(1)].index
stop_df = spacetop_data[spacetop_data['fmri_trigger'] < spacetop_data['fmri_trigger'].shift(1)].index
print(start_df)
print(stop_df)

df_transition = pd.DataFrame({
                        'start_df': start_df, 
                        'stop_df': stop_df
                        })
run_subset = spacetop_data[df_transition.start_df[run_num]: df_transition.stop_df[run_num]]
run_df = run_subset.reset_index()

expect_start = run_df[run_df['expect'] > run_df['expect'].shift(1)]
expect_end = run_df[run_df['expect'] < run_df['expect'].shift(1)]

expect_start_list = expect_start.index.tolist()
print(expect_start_list)
expect_end_list = expect_end.index.tolist()
event_labels = [1,2,1,3,5,6,5,4,4,6,2,3] 


expect_events = nk.events_create(event_onsets=expect_start_list, 
                                 event_durations = 8000, # brauche ich die Länge überhaupt?
                                 event_conditions=event_labels)
print(expect_events)

### Process EDA signal 
note: 
- all these steps are included in nk.eda_process and also in nk.bio_process
- nk.eda_process also includes "only" filters or smoothing, depending on the method
- with subdiving into low-level functions, we have more control over parameters like: 
    - filters
    - minimum amplitude for SCRs (related to specific stimulus, not non-specific SCRs which appear in every eda signal but don't have to be related to a stimulus)

processing steps include: 
- sanitize signal 
    - returns a default indexed signal (really necessary?)
- filter signal (comparable to nk.eda_clean)
    - returns array with filtered signal 
    - smoothing or low-pass filtering (e.g., Butterworth filter) recommended to remove high-frequency noise and small artifacts
- decompose signal into phasic & tonic component 
    - suggested by neurokit documentation -> helps to provide a more accurate estimation of the true SCR amplitude
    - signal is furthermore standardized which is also suggested by neurokit documentation -> useful in presence of high inter-individual variations, which usually is the case for pain perception
- find peaks 
    - returns dict with amplitude of SCR, samples at which SCR onset and SCR peaks occur (accessible with "SCR_Amplitude", "SCR_Onsets", "SCR_Peaks")
- store signal 

In [ ]:
eda_signal = nk.signal_sanitize(run_df["Skin Conductance (EDA) - EDA100C-MRI"])

eda_filters = nk.signal_filter(eda_signal, 
                               sampling_rate=spacetop_samplingrate, 
                               highcut=1, method="butterworth", order=2)
eda_filters_plot = plt.plot(eda_filters)
plt.show(eda_filters_plot)
eda_raw_plot = plt.plot(run_df["Skin Conductance (EDA) - EDA100C-MRI"])
plt.show(eda_raw_plot)

eda_decomposed = nk.eda_phasic(nk.standardize(eda_filters), 
                               sampling_rate=spacetop_samplingrate) 
eda_decomposed_plot = eda_decomposed.plot()

eda_peaks, info = nk.eda_peaks(eda_decomposed["EDA_Phasic"].values,
                               sampling_rate=spacetop_samplingrate, 
                               method = "neurokit", amplitude_min = 0.02)  
info["sampling_rate"] = spacetop_samplingrate

signals = pd.DataFrame({"EDA_Raw": eda_signal, "EDA_Clean": eda_filters})
eda_processed = pd.concat([signals, eda_decomposed, eda_peaks], axis=1) 

### Define epochs for EDA
note: this is still the old code and not updated with Heejung's code yet

In [ ]:
eda_epochs = nk.epochs_create(eda_processed, 
                              expect_events, 
                              sampling_rate=spacetop_samplingrate, 
                              epochs_start=-1, epochs_end=8) # kann man auch end_list nehmen?
plot_epochs_expect = nk.epochs_plot(eda_epochs)
for epoch in eda_epochs.values():
    nk.signal_plot(epoch[["EDA_Clean", "SCR_Height"]], 
    title = epoch['Condition'].values[0],
    standardize = True) 

### Analyze EDA signal 
- eda phasic 
    - event-related anaylsis
    - of interest especially when analyzing EDA responses to specific stimuli 
    - returns: 
        - EDA_SCR: Skin Conductance Response yes (1) or no (0) -> if yes, corresponding components are listed 
        - EDA_Peak_Amplitude: maximum amplitude of phasic component of signal 
        - SCR_Peak_Amplitude: peak amplitude of the first SCR in each epoch (parameter of interest if related to specific stimulus)
        - SCR_Peak_Amplitude_Time: timpepoint of each first SCR peak amplitude (peak should occur within 2-7 s after stimulus onset)
        - SCR_RiseTime: risetime of each first SCR (time it takes for SCR to reach peak amplitude from onset) 
        - SCR_RecoveryTime: half-recovery time of each first SCR (time it takes for SCR to decrease to half amplitude)
- eda tonic 
    - interval-related anaylsis
    - of interest for longer time periods, also resting-state (input could therefore also be a whole condition instead of snipped-out epochs)
    - returns: 
        - SCR_Peaks_N: number of occurrences of Skin Conductance Response 
        - SCR_Peaks_Amplitude_Mean: mean amplitude of SCR peak occurrences 

In [ ]:
eda_phasic = nk.eda_eventrelated(eda_epochs)
eda_tonic = nk.eda_intervalrelated(eda_epochs)

### EDA plots


In [ ]:
eda_plot = nk.eda_plot(eda_processed)
eda_processed_plot = eda_processed.plot() 

eda_features = [info["SCR_Onsets"], info["SCR_Peaks"], info["SCR_Recovery"]]
eda_features_plot = nk.events_plot(eda_features, eda_signal, color= ['red', 'blue', 'orange']) 

### Process PPG signal 
note: 
- all these steps are included in nk.ppg_process and also in nk.bio_process
    - follows exactly the Elgendi method in nk.ppg_process, but with this approach we could always adjust filter parameters 
- with subdiving into low-level functions, we have more control over parameters like: 
    - filters

processing steps include: 
- sanitize signal 
    - returns a default indexed signal (really necessary?)
- filter signal (comparable to nk.ppg_clean)
    - returns array with filtered signal 
- find peaks 
    - returns dict with the samples at which systolic peaks occur (accessible with "PPG_Peaks")
- mark peaks 
- compute rate 
    - signal rate from series of peaks (60/period)
- store signal 

In [ ]:
ppg_signal = nk.signal_sanitize(run_df["Pulse (PPG) - PPG100C"])

ppg_filters = nk.signal_filter(ppg_signal, 
                               sampling_rate=spacetop_samplingrate, 
                               highcut=8, lowcut = 0.5, method="butter_ba", order=3)
plot_ppg_filters = plt.plot(ppg_filters)
plt.show(plot_ppg_filters)
plot_ppg_raw = plt.plot(run_df["Pulse (PPG) - PPG100C"])
plt.show(plot_ppg_raw)

ppg_peaks = nk.ppg_findpeaks(ppg_filters,
                             sampling_rate = spacetop_samplingrate, 
                             show = True)  
info["sampling_rate"] = spacetop_samplingrate

from neurokit2.signal.signal_formatpeaks import _signal_from_indices
ppg_markedpeaks = _signal_from_indices(ppg_peaks["PPG_Peaks"], desired_length = len(ppg_filters))
ppg_markedpeaks.plot()

from neurokit2.signal import signal_rate
ppg_rate = nk.signal_rate(ppg_peaks["PPG_Peaks"], 
                          sampling_rate= spacetop_samplingrate, 
                          desired_length = len(ppg_filters))
ppg_rate_plot = nk.signal_plot(ppg_rate)

ppg_processed = pd.DataFrame(
        {
            "PPG_Raw": ppg_signal,
            "PPG_Clean": ppg_filters,
            "PPG_Rate": ppg_rate,
            "PPG_Peaks": ppg_markedpeaks,
        }
    )

### Define epochs for PPG
note: this is still the old code and not updated with Heejung's code yet

In [ ]:
ppg_epochs = nk.epochs_create(ppg_processed, 
                              expect_events, 
                              sampling_rate=spacetop_samplingrate, 
                              epochs_start=-1, epochs_end=8) # kann man auch end_list nehmen?
plot_ppg_epochs = nk.epochs_plot(ppg_epochs)

### Analyze PPG signal 
- ppg phasic 
    - event-related anaylsis
    - of interest especially when analyzing EDA responses to specific stimuli 
    - returns: 
        - PPG_Rate_Baseline: baseline heart rate (at stimulus onset) 
        - PPG_Rate_Max: maximum heart rate after stimulus onset 
        - PPG_Rate_Min: minimum heart rate after stimulus onset 
        - PPG_Rate_Mean: mean heart rate after stimulus onset 
        - PPG_Rate_SD: standard deviation of the heart rate after stimulus onset
        - PPG_Rate_Max_Time: time at which maximum heart rate occurs 
        - PPG_Rate_Min_Time: time at which minimum heart rate occurs 
        - PPG_Rate_Trend_Linear: parameter corresponding to linear trend 
        - PPG_Rate_Trend_Quadratic: parameter corresponding to curvature 
        - PPG_Rate_Trend_R2: quality of quadratic model, if too low parameters might not be reliable or meaningful 
- ppg tonic -> not working yet, but really necessary? 
    - interval-related anaylsis
    - of interest for longer time periods, also resting-state (input could therefore also be a whole condition instead of snipped-out epochs)
    - returns: 
        - HRV (but makes sense only if time interval is longer than 1 min)

In [ ]:
ppg_phasic = nk.ppg_eventrelated(ppg_epochs)
ppg_tonic = nk.ppg_intervalrelated(ppg_epochs) # error message  

### PPG plots

In [ ]:
ppg_plot = nk.ppg_plot(ppg_processed)